In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

%reload_ext dotenv

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# LOGIN

In [3]:
url = '''https://www.linkedin.com/company/soujunior/posts/?feedView=all'''
driver.get(url)

In [4]:
xpath_login = "/html/body/div/main/div[2]/div[1]/form/div[1]/input"
xpath_password = "/html/body/div/main/div[2]/div[1]/form/div[2]/input"
xpath_submit = "/html/body/div/main/div[2]/div[1]/form/div[3]/button"

login_linkedin = os.getenv("LOGIN_LINKEDIN")
password_linkedin = os.getenv("PASSWORD_LINKEDIN")

In [5]:
input_login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_login)))
input_password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_password)))
input_submit = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_submit)))

In [6]:
input_login.clear()
input_password.clear()

input_login.send_keys(login_linkedin)
input_password.send_keys(password_linkedin)
input_submit.click()

# SCRAP

In [26]:
body = driver.find_element(By.TAG_NAME, "body")

for _ in range(20):
    body.send_keys(Keys.PAGE_DOWN)

    time.sleep(2)

In [29]:
xpath_feed_posts = "/html/body/div[5]/div[3]/div/div[2]/div/div[2]/main/div[2]/div/div[2]/div[3]/div/div[1]"

feed_posts = driver.find_element('xpath', xpath_feed_posts)

posts = feed_posts.find_elements(By.XPATH, './div')

print(len(posts))

12


In [15]:
#printando a imagem do autor
img = posts[0].find_element(By.CLASS_NAME, 'update-components-actor__avatar-image')

#salvando a imagem
img.screenshot("screenshot.png")

True

In [33]:
name = posts[4].find_element(By.CLASS_NAME, "update-components-actor__name")
name = name.find_element(By.CLASS_NAME, "visually-hidden").text
name

'Dener Garcia 🧑🏻\u200d💻'

In [46]:
#individual
for index, post in enumerate(posts):
    author = []
    author = post.find_elements(By.CLASS_NAME, "update-components-actor__name")

    # validation
    if author:
        author_name = author[0].find_element(By.CLASS_NAME, "visually-hidden").text
    else:
        continue

    img = post.find_element(By.CLASS_NAME, 'update-components-actor__avatar-image')

    # save post author image
    img_path = f"linkedin_page/scraps/03/profile_pics/{author_name}.png"
    if not os.path.exists(img_path):
        img.screenshot(f"linkedin_page/scraps/03/profile_pics/{author_name}.png")

    # save post
    with open(f'linkedin_page/scraps/03/post_{index}.html', 'w', encoding='utf-8') as file:
        file.write(f'<p>{author_name}</p>\n')
        file.write(post.get_attribute('outerHTML'))